In [13]:
#pip install --upgrade transformers tensorflow pandas scikit-learn tf-keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 2.9 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 4.5 MB/s eta 0:00:0000:0100:01


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

# Load the cleaned data
df = pd.read_csv('cleaned_tweets.csv')

# Ensure all entries in 'cleaned_text' are strings
df['cleaned_text'] = df['cleaned_text'].astype(str)

# Select 20,000 records (if necessary)
df = df.sample(n=5000, random_state=42)

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['cleaned_text'], df['target'], test_size=0.2, random_state=42)


In [2]:
# Load DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [3]:
# Tokenize the texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

# Convert encodings to TensorFlow datasets
def encode_data(encodings, labels):
    input_ids = tf.convert_to_tensor(encodings['input_ids'])
    attention_masks = tf.convert_to_tensor(encodings['attention_mask'])
    labels = tf.convert_to_tensor(labels)
    return tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_masks}, labels))

train_dataset = encode_data(train_encodings, train_labels)
test_dataset = encode_data(test_encodings, test_labels)

train_dataset = train_dataset.shuffle(10000).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)


In [4]:
# Load the pre-trained DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=3, validation_data=test_dataset)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
250/250 [==============================] - 183s 687ms/step - loss: 0.5641 - accuracy: 0.7065 - val_loss: 0.5459 - val_accuracy: 0.7260
Epoch 2/3
250/250 [==============================] - 183s 733ms/step - loss: 0.3945 - accuracy: 0.8285 - val_loss: 0.5576 - val_accuracy: 0.7360
Epoch 3/3
250/250 [==============================] - 178s 713ms/step - loss: 0.2105 - accuracy: 0.9190 - val_loss: 0.6865 - val_accuracy: 0.7380


In [6]:
# Sample tweets
sample_tweets = ["The movie was very touching and heart whelming", 
            "I have never seen an awesome movie like this", 
            "the movie plot was great but it had terrible acting",
           "I thought the movie would be boring",
           "I thought the movie would be boring, but it was surprisingly good!",
               "The movie was an absolute masterpiece, with stunning visuals and a compelling storyline.",
    "I loved every minute of the film; the acting was top-notch, and the soundtrack was beautiful.",
    "The plot was incredibly dull, and the characters were poorly developed. A complete waste of time.",
    "I found the movie to be a predictable and uninspired rehash of better films.",
    "The movie had an interesting concept, but the execution fell flat, and the pacing was all over the place.",
    "While the cinematography was breathtaking, the plot was convoluted and hard to follow.",
    "This film exceeded all my expectations with its brilliant script and stellar performances. A must-watch!",
    "An emotionally gripping story that kept me on the edge of my seat. Highly recommend it.",
    "The special effects were outdated, and the dialogue was cringeworthy. I couldn't wait for it to end.",
    "Despite a few decent scenes, the overall experience was boring and forgettable.",
    "The first half of the movie was engaging, but it lost momentum in the second half and ended on a weak note.",
    "I appreciated the artistic direction, but the acting was subpar, and the story lacked coherence."
]
# Preprocess new tweets
sample_encodings = tokenizer(sample_tweets, truncation=True, padding=True, max_length=128)
sample_dataset = tf.data.Dataset.from_tensor_slices((
    dict(sample_encodings)
)).batch(1)

# Predict
predictions = model.predict(sample_dataset).logits
predicted_labels = tf.argmax(predictions, axis=1).numpy()
print("Predictions:", predicted_labels)

17/17 [==============================] - 1s 19ms/step
Predictions: [1 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0]
